<a href="https://colab.research.google.com/github/YuriCha02/TP1_CodsState_16/blob/main/AnnModel3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%run /content/AnnModel1.ipynb

@차지형

**설명**

`AnnModel1.ipynb`파일내부에 method를 사용할수 있게, 실행함

In [ ]:
# 메서드 정의
def multiple_load_dataset():
    with open('.../mulit_classification_data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt

    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

@차지형

**Multiple_load_dataset()에 결과 값**

data, input_cnt, output_cnt를 글로벌 변수값으로 저장해서, 다른 변수에서 사용이 가능함.
- 대신 변수 이름이 겹칠 가능성이 높음.
  - 특히 고객들이 코드를 편집해서 사용해야 할 경우

`data`:

- 행:
> ```
[ 4.20000e+01  5.00000e+01  2.70900e+05  2.70944e+05  2.67000e+02
  1.70000e+01  4.40000e+01  2.42200e+04  7.60000e+01  1.08000e+02
  1.68700e+03  1.00000e+00  0.00000e+00  8.00000e+01  4.98000e-02
  2.41500e-01  1.81800e-01  4.70000e-03  4.70600e-01  1.00000e+00
  1.00000e+00  2.42650e+00  9.03100e-01  1.64350e+00  8.18200e-01
 -2.91300e-01  5.82200e-01  1.00000e+00  0.00000e+00  0.00000e+00
  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00]
```
- 열 길이: 34
- 행 길이: 1941

In [ ]:
# 메서드 정의
def multiple_main(epoch_count = 10, mb_size = 10, report=1, train_ratio = 0.6):
    multiple_load_dataset()
    init_param()
    train_and_test(epoch_count,mb_size,report,train_ratio)

@차지형

**설명**

- `ANNModel1.ipynb`에서 `init_param()`과 `train_and_test()`을 불러옴
- `multiple_load_dataset()`로 데이터를 불러와,
 - `data`, `input_cnt`, `output_cnt` 글로벌 변수를 출력함
  - `input_cnt`, `output_cnt`는 AnnModel1에서 가져온 메서드 `init_param()`에 사용됨

In [ ]:
# 메서드 정의
def softmax(x):

    max_elem = np.max(x, axis = 1)

    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)

    sum_exp = np.sum(exp, axis = 1)

    probs = (exp.transpose()/sum_exp).transpose()

    return probs

@차지형

**코드 설명:**

`max_elem = np.max(x, axis = 1)`: 각 행에서 최대 값을 구하여, `max_elem`에 저장함

`diff = (x.transpose() - max_elem).transpose()`: 각 행의 모든 값과 그 행의 차이를 구하여 `diff`에 저장함.
- 아마 다름 `np.exp()`에서 negative 값을 positive 값으로 바꿀떄,값들 사이에 차이가 너무 벌어지지 않도록 차이를 구하서 거리를 좁히는 것 같음

`exp = np.exp(diff)`: negative 값을 positve값으로 변경하여 `exp`로 저장함

`sum_exp = np.sum(exp, axis = 1)`: 각 행에서 값들을 모두 더해서 `sum_exp`에 저장함

`probs = (exp.transpose()/sum_exp).transpose()`: `exp`에서 'sum_exp'를 나누어 확률을 구함

`return probs`: 확률을 function 외부로 배출함

**추가**: pytorch를 사용할 경우, `transpose()`를 일일히 사용할 필요 없이, 내부에서 알아서 해줌.

In [ ]:
# 메서드 정의
def softmax_cross_entropy(y_real, y_hat):

    probs = softmax(y_hat)

    #기존 확률값에 아주 작은 양수를 더해 log함수의 폭주 방지
    return - np.sum(y_real * np.log(probs+1.0e-10), axis=1)

@차지형

**설명**

`y_real`: test 확률

`y_hat`: 예측확률


`probs = softmax(y_hat)`예측확률을 softmax 확률로 변경하여 `probs`에 저장함

`return - np.sum(y_real * np.log(probs+1.0e-10), axis=1)`: 각 행 (or 배치)에서 cross-entropy loss 값을 구함

In [ ]:
# 메서드 정의
def forward_postproc(y_hat, y_real):
    entropy  = softmax_cross_entropy(y_real, y_hat)
    loss     = np.mean(entropy)

    return loss, [y_real, y_hat, entropy]

@차지형

**설명**

`entropy  = softmax_cross_entropy(y_real, y_hat)`: 예측값과 테스트 값을 비교하여 cross_entropy loss 값을 구하여 `entropy`로 저장함

`loss = np.mean(entropy)`: `entropy`의 평균 값을 구함

`return loss, [y_real, y_hat, entropy]`:
- 평균 loss 점수을 출력함
- 실제값과 예측값, 그리고 cross-entropy loss 값을 리스트로 출력함

In [ ]:
# 메서드 정의
def softmax_cross_entropy_with_derv(y_real,y_hat):

    return softmax(y_hat) - y_real

@차지형

**설명**

`return softmax(y_hat) - y_real`: 예측값의 softmax 확율과 실제값의 차이로 기울기를 구해 외부로 출력함.

In [ ]:
# 메서드 정의
def backprop_postproc(aux_pp):
    y_real, y_hat, entropy = aux_pp

    g_loss_entropy = 1.0/np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_derv(y_real,y_hat)

    G_output = g_entropy_output * g_loss_entropy

    return G_output

@차지형

**설명**

`aux_pp`: 위에 `forward_postproc` 메소드에서 출력한 `[y_real, y_hat, entropy]` 값임

`y_real, y_hat, entropy = aux_pp`: 리스트를 다시 각각 나눔.

`g_loss_entropy = 1.0/np.prod(entropy.shape)`: cross_entropy loss 값을 normalization 함
- 예측값에 맞추어 모델의 기울기를 조절해주기 위해 만든 변수라고 생각함.

`g_entropy_output = softmax_cross_entropy_with_derv(y_real,y_hat)`: 위에 정의된 `softmax_cross_entropy_with_derv` 메소드를 사용하여 모델의 기울기를 `g_entropy_output`라는 변수에 저장함.

`G_output = g_entropy_output * g_loss_entropy`: 모델의 예측에 대한 손실의 기울기를 계산함

`return G_output`: 모델의 손실 기울기를 외부로 출력함.



In [ ]:
# 메서드 정의
def eval_accuracy(y_hat,y_real):
    estimate = np.argmax(y_hat,axis=1)
    answer = np.argmax(y_real,axis=1)

    correct = np.equal(estimate, answer)

    return np.mean(correct)

@ 차지형

**설명**

`estimate = np.argmax(y_hat,axis=1)`: 예측확률 중에서 가장 가능성이 높은 레벨을 선택하여 `estimate`라는 변수에 저장함

`answer = np.argmax(y_real,axis=1)`: 실제확률 중에서 가장 가능성이 높은 레벨을 선택하여 `answer`라는 변수에 저장함

`correct = np.equal(estimate, answer)`: `estimate`과 `answer`를 비교하여, 값이 같으면 `True`를, 다른면 `False`를 `correct`라는 변수에 저장합니다.

`return np.mean(correct)`: `correct`내의 `True`를 기준으로 평균을 외부로 출력합니다.